In [161]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# Noisy Variables Removed

In [162]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'], axis=1)

In [166]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [167]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than ...?
replace_application = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

In [168]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

In [169]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)

# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True,right_index=True)
application_df = application_df.drop(application_cat,axis=1)
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [170]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Attempt 1: More Neurons

In [180]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 26
hidden_nodes_layer2 = 14

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_65 (Dense)            (None, 26)                1066      
                                                                 
 dense_66 (Dense)            (None, 14)                378       
                                                                 
 dense_67 (Dense)            (None, 1)                 15        
                                                                 
Total params: 1,459
Trainable params: 1,459
Non-trainable params: 0
_________________________________________________________________


In [181]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 684us/step - loss: 0.5771 - accuracy: 0.7154
Epoch 2/100
804/804 [==============================] - 1s 676us/step - loss: 0.5568 - accuracy: 0.7273
Epoch 3/100
804/804 [==============================] - 1s 677us/step - loss: 0.5537 - accuracy: 0.7283
Epoch 4/100
804/804 [==============================] - 1s 677us/step - loss: 0.5518 - accuracy: 0.7308
Epoch 5/100
804/804 [==============================] - 1s 677us/step - loss: 0.5503 - accuracy: 0.7311
Epoch 6/100
804/804 [==============================] - 1s 676us/step - loss: 0.5484 - accuracy: 0.7327
Epoch 7/100
804/804 [==============================] - 1s 693us/step - loss: 0.5477 - accuracy: 0.7317
Epoch 8/100
804/804 [==============================] - 1s 680us/step - loss: 0.5469 - accuracy: 0.7327
Epoch 9/100
804/804 [==============================] - 1s 680us/step - loss: 0.5466 - accuracy: 0.7323
Epoch 10/100
804/804 [==============================] - 1s 684us/step - l

804/804 [==============================] - 1s 681us/step - loss: 0.5381 - accuracy: 0.7389
Epoch 80/100
804/804 [==============================] - 1s 684us/step - loss: 0.5376 - accuracy: 0.7380
Epoch 81/100
804/804 [==============================] - 1s 676us/step - loss: 0.5376 - accuracy: 0.7377
Epoch 82/100
804/804 [==============================] - 1s 673us/step - loss: 0.5377 - accuracy: 0.7388
Epoch 83/100
804/804 [==============================] - 1s 685us/step - loss: 0.5376 - accuracy: 0.7380
Epoch 84/100
804/804 [==============================] - 1s 677us/step - loss: 0.5370 - accuracy: 0.7381
Epoch 85/100
804/804 [==============================] - 1s 674us/step - loss: 0.5376 - accuracy: 0.7368
Epoch 86/100
804/804 [==============================] - 1s 692us/step - loss: 0.5372 - accuracy: 0.7381
Epoch 87/100
804/804 [==============================] - 1s 714us/step - loss: 0.5375 - accuracy: 0.7370
Epoch 88/100
804/804 [==============================] - 1s 677us/step - loss:

In [182]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5553 - accuracy: 0.7285 - 230ms/epoch - 859us/step
Loss: 0.5553306937217712, Accuracy: 0.7285131216049194


# Attempt 2: More Epochs

In [183]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 26
hidden_nodes_layer2 = 14

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_68 (Dense)            (None, 26)                1066      
                                                                 
 dense_69 (Dense)            (None, 14)                378       
                                                                 
 dense_70 (Dense)            (None, 1)                 15        
                                                                 
Total params: 1,459
Trainable params: 1,459
Non-trainable params: 0
_________________________________________________________________


In [184]:
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_2.fit(X_train_scaled,y_train,epochs=500)

Epoch 1/500
804/804 [==============================] - 1s 699us/step - loss: 0.5834 - accuracy: 0.7109
Epoch 2/500
804/804 [==============================] - 1s 687us/step - loss: 0.5554 - accuracy: 0.7280
Epoch 3/500
804/804 [==============================] - 1s 696us/step - loss: 0.5523 - accuracy: 0.7303
Epoch 4/500
804/804 [==============================] - 1s 685us/step - loss: 0.5510 - accuracy: 0.7299
Epoch 5/500
804/804 [==============================] - 1s 692us/step - loss: 0.5493 - accuracy: 0.7306
Epoch 6/500
804/804 [==============================] - 1s 680us/step - loss: 0.5483 - accuracy: 0.7310
Epoch 7/500
804/804 [==============================] - 1s 704us/step - loss: 0.5478 - accuracy: 0.7312
Epoch 8/500
804/804 [==============================] - 1s 679us/step - loss: 0.5468 - accuracy: 0.7334
Epoch 9/500
804/804 [==============================] - 1s 694us/step - loss: 0.5463 - accuracy: 0.7329
Epoch 10/500
804/804 [==============================] - 1s 707us/step - l

804/804 [==============================] - 1s 681us/step - loss: 0.5376 - accuracy: 0.7381
Epoch 80/500
804/804 [==============================] - 1s 694us/step - loss: 0.5376 - accuracy: 0.7385
Epoch 81/500
804/804 [==============================] - 1s 861us/step - loss: 0.5378 - accuracy: 0.7384
Epoch 82/500
804/804 [==============================] - 1s 732us/step - loss: 0.5380 - accuracy: 0.7374
Epoch 83/500
804/804 [==============================] - 1s 768us/step - loss: 0.5377 - accuracy: 0.7376
Epoch 84/500
804/804 [==============================] - 1s 749us/step - loss: 0.5376 - accuracy: 0.7388
Epoch 85/500
804/804 [==============================] - 1s 832us/step - loss: 0.5377 - accuracy: 0.7372
Epoch 86/500
804/804 [==============================] - 1s 860us/step - loss: 0.5373 - accuracy: 0.7382
Epoch 87/500
804/804 [==============================] - 1s 799us/step - loss: 0.5376 - accuracy: 0.7381
Epoch 88/500
804/804 [==============================] - 1s 861us/step - loss:

804/804 [==============================] - 1s 665us/step - loss: 0.5358 - accuracy: 0.7394
Epoch 158/500
804/804 [==============================] - 1s 661us/step - loss: 0.5356 - accuracy: 0.7390
Epoch 159/500
804/804 [==============================] - 1s 664us/step - loss: 0.5357 - accuracy: 0.7385
Epoch 160/500
804/804 [==============================] - 1s 673us/step - loss: 0.5352 - accuracy: 0.7384
Epoch 161/500
804/804 [==============================] - 1s 668us/step - loss: 0.5359 - accuracy: 0.7381
Epoch 162/500
804/804 [==============================] - 1s 669us/step - loss: 0.5354 - accuracy: 0.7374
Epoch 163/500
804/804 [==============================] - 1s 666us/step - loss: 0.5353 - accuracy: 0.7388
Epoch 164/500
804/804 [==============================] - 1s 670us/step - loss: 0.5354 - accuracy: 0.7390
Epoch 165/500
804/804 [==============================] - 1s 676us/step - loss: 0.5354 - accuracy: 0.7391
Epoch 166/500
804/804 [==============================] - 1s 677us/ste

804/804 [==============================] - 1s 703us/step - loss: 0.5349 - accuracy: 0.7382
Epoch 236/500
804/804 [==============================] - 1s 677us/step - loss: 0.5345 - accuracy: 0.7395
Epoch 237/500
804/804 [==============================] - 1s 679us/step - loss: 0.5347 - accuracy: 0.7395
Epoch 238/500
804/804 [==============================] - 1s 671us/step - loss: 0.5346 - accuracy: 0.7399
Epoch 239/500
804/804 [==============================] - 1s 671us/step - loss: 0.5343 - accuracy: 0.7395
Epoch 240/500
804/804 [==============================] - 1s 670us/step - loss: 0.5348 - accuracy: 0.7386
Epoch 241/500
804/804 [==============================] - 1s 690us/step - loss: 0.5344 - accuracy: 0.7406
Epoch 242/500
804/804 [==============================] - 1s 673us/step - loss: 0.5346 - accuracy: 0.7387
Epoch 243/500
804/804 [==============================] - 1s 675us/step - loss: 0.5345 - accuracy: 0.7388
Epoch 244/500
804/804 [==============================] - 1s 677us/ste

804/804 [==============================] - 1s 669us/step - loss: 0.5341 - accuracy: 0.7394
Epoch 314/500
804/804 [==============================] - 1s 666us/step - loss: 0.5337 - accuracy: 0.7399
Epoch 315/500
804/804 [==============================] - 1s 670us/step - loss: 0.5339 - accuracy: 0.7399
Epoch 316/500
804/804 [==============================] - 1s 673us/step - loss: 0.5338 - accuracy: 0.7403
Epoch 317/500
804/804 [==============================] - 1s 666us/step - loss: 0.5338 - accuracy: 0.7393
Epoch 318/500
804/804 [==============================] - 1s 668us/step - loss: 0.5338 - accuracy: 0.7408
Epoch 319/500
804/804 [==============================] - 1s 659us/step - loss: 0.5339 - accuracy: 0.7392
Epoch 320/500
804/804 [==============================] - 1s 674us/step - loss: 0.5340 - accuracy: 0.7395
Epoch 321/500
804/804 [==============================] - 1s 675us/step - loss: 0.5340 - accuracy: 0.7394
Epoch 322/500
804/804 [==============================] - 1s 673us/ste

804/804 [==============================] - 1s 665us/step - loss: 0.5336 - accuracy: 0.7397
Epoch 392/500
804/804 [==============================] - 1s 664us/step - loss: 0.5335 - accuracy: 0.7392
Epoch 393/500
804/804 [==============================] - 1s 664us/step - loss: 0.5336 - accuracy: 0.7397
Epoch 394/500
804/804 [==============================] - 1s 677us/step - loss: 0.5336 - accuracy: 0.7400
Epoch 395/500
804/804 [==============================] - 1s 673us/step - loss: 0.5334 - accuracy: 0.7387
Epoch 396/500
804/804 [==============================] - 1s 671us/step - loss: 0.5334 - accuracy: 0.7403
Epoch 397/500
804/804 [==============================] - 1s 671us/step - loss: 0.5332 - accuracy: 0.7407
Epoch 398/500
804/804 [==============================] - 1s 672us/step - loss: 0.5336 - accuracy: 0.7397
Epoch 399/500
804/804 [==============================] - 1s 672us/step - loss: 0.5336 - accuracy: 0.7392
Epoch 400/500
804/804 [==============================] - 1s 671us/ste

804/804 [==============================] - 1s 691us/step - loss: 0.5332 - accuracy: 0.7405
Epoch 470/500
804/804 [==============================] - 1s 688us/step - loss: 0.5333 - accuracy: 0.7399
Epoch 471/500
804/804 [==============================] - 1s 691us/step - loss: 0.5332 - accuracy: 0.7407
Epoch 472/500
804/804 [==============================] - 1s 675us/step - loss: 0.5329 - accuracy: 0.7394
Epoch 473/500
804/804 [==============================] - 1s 689us/step - loss: 0.5330 - accuracy: 0.7399
Epoch 474/500
804/804 [==============================] - 1s 683us/step - loss: 0.5334 - accuracy: 0.7401
Epoch 475/500
804/804 [==============================] - 1s 669us/step - loss: 0.5331 - accuracy: 0.7400
Epoch 476/500
804/804 [==============================] - 1s 695us/step - loss: 0.5328 - accuracy: 0.7399
Epoch 477/500
804/804 [==============================] - 1s 680us/step - loss: 0.5329 - accuracy: 0.7420
Epoch 478/500
804/804 [==============================] - 1s 685us/ste

In [185]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5643 - accuracy: 0.7282 - 223ms/epoch - 833us/step
Loss: 0.5642739534378052, Accuracy: 0.7281632423400879


# Attempt 3: Different Activation Functions

In [186]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 26
hidden_nodes_layer2 = 14

nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_3.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_71 (Dense)            (None, 26)                1066      
                                                                 
 dense_72 (Dense)            (None, 14)                378       
                                                                 
 dense_73 (Dense)            (None, 1)                 15        
                                                                 
Total params: 1,459
Trainable params: 1,459
Non-trainable params: 0
_________________________________________________________________


In [187]:
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_3.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 692us/step - loss: 0.5821 - accuracy: 0.7091
Epoch 2/100
804/804 [==============================] - 1s 696us/step - loss: 0.5564 - accuracy: 0.7292
Epoch 3/100
804/804 [==============================] - 1s 680us/step - loss: 0.5524 - accuracy: 0.7299
Epoch 4/100
804/804 [==============================] - 1s 677us/step - loss: 0.5504 - accuracy: 0.7309
Epoch 5/100
804/804 [==============================] - 1s 691us/step - loss: 0.5490 - accuracy: 0.7321
Epoch 6/100
804/804 [==============================] - 1s 673us/step - loss: 0.5482 - accuracy: 0.7317
Epoch 7/100
804/804 [==============================] - 1s 674us/step - loss: 0.5472 - accuracy: 0.7318
Epoch 8/100
804/804 [==============================] - 1s 677us/step - loss: 0.5467 - accuracy: 0.7321
Epoch 9/100
804/804 [==============================] - 1s 678us/step - loss: 0.5459 - accuracy: 0.7335
Epoch 10/100
804/804 [==============================] - 1s 674us/step - l

804/804 [==============================] - 1s 677us/step - loss: 0.5368 - accuracy: 0.7384
Epoch 80/100
804/804 [==============================] - 1s 669us/step - loss: 0.5369 - accuracy: 0.7386
Epoch 81/100
804/804 [==============================] - 1s 663us/step - loss: 0.5370 - accuracy: 0.7388
Epoch 82/100
804/804 [==============================] - 1s 664us/step - loss: 0.5367 - accuracy: 0.7390
Epoch 83/100
804/804 [==============================] - 1s 661us/step - loss: 0.5372 - accuracy: 0.7380
Epoch 84/100
804/804 [==============================] - 1s 666us/step - loss: 0.5369 - accuracy: 0.7386
Epoch 85/100
804/804 [==============================] - 1s 658us/step - loss: 0.5369 - accuracy: 0.7388
Epoch 86/100
804/804 [==============================] - 1s 670us/step - loss: 0.5367 - accuracy: 0.7374
Epoch 87/100
804/804 [==============================] - 1s 673us/step - loss: 0.5369 - accuracy: 0.7385
Epoch 88/100
804/804 [==============================] - 1s 682us/step - loss:

In [188]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5574 - accuracy: 0.7261 - 216ms/epoch - 807us/step
Loss: 0.5573692917823792, Accuracy: 0.726064145565033


# Attempt 4: More Hidden Layers

In [189]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 22
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 6

nn_4 = tf.keras.models.Sequential()

# First hidden layer
nn_4.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_4.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_4.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_74 (Dense)            (None, 22)                902       
                                                                 
 dense_75 (Dense)            (None, 12)                276       
                                                                 
 dense_76 (Dense)            (None, 6)                 78        
                                                                 
 dense_77 (Dense)            (None, 1)                 7         
                                                                 
Total params: 1,263
Trainable params: 1,263
Non-trainable params: 0
_________________________________________________________________


In [190]:
nn_4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_4.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 739us/step - loss: 0.5791 - accuracy: 0.7141
Epoch 2/100
804/804 [==============================] - 1s 709us/step - loss: 0.5552 - accuracy: 0.7276
Epoch 3/100
804/804 [==============================] - 1s 712us/step - loss: 0.5529 - accuracy: 0.7296
Epoch 4/100
804/804 [==============================] - 1s 712us/step - loss: 0.5503 - accuracy: 0.7304
Epoch 5/100
804/804 [==============================] - 1s 711us/step - loss: 0.5495 - accuracy: 0.7300
Epoch 6/100
804/804 [==============================] - 1s 712us/step - loss: 0.5486 - accuracy: 0.7314
Epoch 7/100
804/804 [==============================] - 1s 722us/step - loss: 0.5479 - accuracy: 0.7312
Epoch 8/100
804/804 [==============================] - 1s 709us/step - loss: 0.5472 - accuracy: 0.7314
Epoch 9/100
804/804 [==============================] - 1s 712us/step - loss: 0.5463 - accuracy: 0.7325
Epoch 10/100
804/804 [==============================] - 1s 706us/step - l

804/804 [==============================] - 1s 718us/step - loss: 0.5382 - accuracy: 0.7374
Epoch 80/100
804/804 [==============================] - 1s 715us/step - loss: 0.5384 - accuracy: 0.7378
Epoch 81/100
804/804 [==============================] - 1s 710us/step - loss: 0.5386 - accuracy: 0.7365
Epoch 82/100
804/804 [==============================] - 1s 716us/step - loss: 0.5383 - accuracy: 0.7376
Epoch 83/100
804/804 [==============================] - 1s 709us/step - loss: 0.5382 - accuracy: 0.7374
Epoch 84/100
804/804 [==============================] - 1s 716us/step - loss: 0.5382 - accuracy: 0.7373
Epoch 85/100
804/804 [==============================] - 1s 708us/step - loss: 0.5380 - accuracy: 0.7374
Epoch 86/100
804/804 [==============================] - 1s 709us/step - loss: 0.5382 - accuracy: 0.7361
Epoch 87/100
804/804 [==============================] - 1s 700us/step - loss: 0.5386 - accuracy: 0.7362
Epoch 88/100
804/804 [==============================] - 1s 701us/step - loss:

In [191]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5545 - accuracy: 0.7299 - 240ms/epoch - 895us/step
Loss: 0.5545274615287781, Accuracy: 0.729912519454956


# Attempt 5: All of the Above

In [192]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 22
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 6

nn_5 = tf.keras.models.Sequential()

# First hidden layer
nn_5.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn_5.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn_5.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_5.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_5.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_78 (Dense)            (None, 22)                902       
                                                                 
 dense_79 (Dense)            (None, 12)                276       
                                                                 
 dense_80 (Dense)            (None, 6)                 78        
                                                                 
 dense_81 (Dense)            (None, 1)                 7         
                                                                 
Total params: 1,263
Trainable params: 1,263
Non-trainable params: 0
_________________________________________________________________


In [193]:
# Compile the model
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/Optimization_Weights.{epoch:02d}.hdf5"

nn_5.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5
)

# Train the model
fit_model = nn_5.fit(X_train_scaled,y_train,epochs=500,callbacks=[cp_callback])

Epoch 1/500
804/804 [==============================] - 1s 724us/step - loss: 0.5772 - accuracy: 0.7131
Epoch 2/500
804/804 [==============================] - 1s 709us/step - loss: 0.5558 - accuracy: 0.7291
Epoch 3/500
804/804 [==============================] - 1s 708us/step - loss: 0.5530 - accuracy: 0.7296
Epoch 4/500
804/804 [==============================] - 1s 709us/step - loss: 0.5503 - accuracy: 0.7317
Epoch 5/500
778/804 [============================>.] - ETA: 0s - loss: 0.5490 - accuracy: 0.7310
Epoch 5: saving model to checkpoints\Optimization_Weights.05.hdf5
804/804 [==============================] - 1s 730us/step - loss: 0.5489 - accuracy: 0.7311
Epoch 6/500
804/804 [==============================] - 1s 718us/step - loss: 0.5482 - accuracy: 0.7312
Epoch 7/500
804/804 [==============================] - 1s 721us/step - loss: 0.5473 - accuracy: 0.7330
Epoch 8/500
804/804 [==============================] - 1s 764us/step - loss: 0.5461 - accuracy: 0.7334
Epoch 9/500
804/804 [====

804/804 [==============================] - 1s 739us/step - loss: 0.5386 - accuracy: 0.7385
Epoch 62/500
804/804 [==============================] - 1s 761us/step - loss: 0.5386 - accuracy: 0.7382
Epoch 63/500
804/804 [==============================] - 1s 734us/step - loss: 0.5384 - accuracy: 0.7379
Epoch 64/500
804/804 [==============================] - 1s 706us/step - loss: 0.5381 - accuracy: 0.7381
Epoch 65/500
778/804 [============================>.] - ETA: 0s - loss: 0.5377 - accuracy: 0.7382
Epoch 65: saving model to checkpoints\Optimization_Weights.65.hdf5
804/804 [==============================] - 1s 717us/step - loss: 0.5383 - accuracy: 0.7382
Epoch 66/500
804/804 [==============================] - 1s 709us/step - loss: 0.5380 - accuracy: 0.7385
Epoch 67/500
804/804 [==============================] - 1s 707us/step - loss: 0.5378 - accuracy: 0.7381
Epoch 68/500
804/804 [==============================] - 1s 734us/step - loss: 0.5381 - accuracy: 0.7379
Epoch 69/500
804/804 [=======

804/804 [==============================] - 1s 703us/step - loss: 0.5359 - accuracy: 0.7397
Epoch 123/500
804/804 [==============================] - 1s 851us/step - loss: 0.5357 - accuracy: 0.7395
Epoch 124/500
804/804 [==============================] - 1s 754us/step - loss: 0.5358 - accuracy: 0.7394
Epoch 125/500
766/804 [===========================>..] - ETA: 0s - loss: 0.5356 - accuracy: 0.7400
Epoch 125: saving model to checkpoints\Optimization_Weights.125.hdf5
804/804 [==============================] - 1s 730us/step - loss: 0.5359 - accuracy: 0.7400
Epoch 126/500
804/804 [==============================] - 1s 734us/step - loss: 0.5354 - accuracy: 0.7390
Epoch 127/500
804/804 [==============================] - 1s 776us/step - loss: 0.5356 - accuracy: 0.7395
Epoch 128/500
804/804 [==============================] - 1s 752us/step - loss: 0.5356 - accuracy: 0.7389
Epoch 129/500
804/804 [==============================] - 1s 704us/step - loss: 0.5357 - accuracy: 0.7396
Epoch 130/500
764/80

804/804 [==============================] - 1s 742us/step - loss: 0.5346 - accuracy: 0.7395
Epoch 183/500
804/804 [==============================] - 1s 713us/step - loss: 0.5344 - accuracy: 0.7410
Epoch 184/500
804/804 [==============================] - 1s 706us/step - loss: 0.5344 - accuracy: 0.7398
Epoch 185/500
776/804 [===========================>..] - ETA: 0s - loss: 0.5340 - accuracy: 0.7402
Epoch 185: saving model to checkpoints\Optimization_Weights.185.hdf5
804/804 [==============================] - 1s 720us/step - loss: 0.5344 - accuracy: 0.7397
Epoch 186/500
804/804 [==============================] - 1s 718us/step - loss: 0.5346 - accuracy: 0.7399
Epoch 187/500
804/804 [==============================] - 1s 714us/step - loss: 0.5346 - accuracy: 0.7397
Epoch 188/500
804/804 [==============================] - 1s 747us/step - loss: 0.5345 - accuracy: 0.7398
Epoch 189/500
804/804 [==============================] - 1s 717us/step - loss: 0.5344 - accuracy: 0.7397
Epoch 190/500
794/80

804/804 [==============================] - 1s 749us/step - loss: 0.5337 - accuracy: 0.7400
Epoch 243/500
804/804 [==============================] - 1s 765us/step - loss: 0.5337 - accuracy: 0.7401
Epoch 244/500
804/804 [==============================] - 1s 715us/step - loss: 0.5337 - accuracy: 0.7408
Epoch 245/500
782/804 [============================>.] - ETA: 0s - loss: 0.5330 - accuracy: 0.7411
Epoch 245: saving model to checkpoints\Optimization_Weights.245.hdf5
804/804 [==============================] - 1s 715us/step - loss: 0.5337 - accuracy: 0.7403
Epoch 246/500
804/804 [==============================] - 1s 710us/step - loss: 0.5336 - accuracy: 0.7404
Epoch 247/500
804/804 [==============================] - 1s 719us/step - loss: 0.5337 - accuracy: 0.7396
Epoch 248/500
804/804 [==============================] - 1s 743us/step - loss: 0.5337 - accuracy: 0.7399
Epoch 249/500
804/804 [==============================] - 1s 709us/step - loss: 0.5334 - accuracy: 0.7398
Epoch 250/500
781/80

804/804 [==============================] - 1s 711us/step - loss: 0.5333 - accuracy: 0.7402
Epoch 303/500
804/804 [==============================] - 1s 734us/step - loss: 0.5331 - accuracy: 0.7407
Epoch 304/500
804/804 [==============================] - 1s 719us/step - loss: 0.5332 - accuracy: 0.7405
Epoch 305/500
756/804 [===========================>..] - ETA: 0s - loss: 0.5337 - accuracy: 0.7404
Epoch 305: saving model to checkpoints\Optimization_Weights.305.hdf5
804/804 [==============================] - 1s 736us/step - loss: 0.5332 - accuracy: 0.7407
Epoch 306/500
804/804 [==============================] - 1s 713us/step - loss: 0.5332 - accuracy: 0.7406
Epoch 307/500
804/804 [==============================] - 1s 708us/step - loss: 0.5331 - accuracy: 0.7411
Epoch 308/500
804/804 [==============================] - 1s 734us/step - loss: 0.5331 - accuracy: 0.7399
Epoch 309/500
804/804 [==============================] - 1s 707us/step - loss: 0.5333 - accuracy: 0.7398
Epoch 310/500
779/80

804/804 [==============================] - 1s 713us/step - loss: 0.5327 - accuracy: 0.7408
Epoch 363/500
804/804 [==============================] - 1s 739us/step - loss: 0.5328 - accuracy: 0.7406
Epoch 364/500
804/804 [==============================] - 1s 708us/step - loss: 0.5327 - accuracy: 0.7406
Epoch 365/500
780/804 [============================>.] - ETA: 0s - loss: 0.5329 - accuracy: 0.7402
Epoch 365: saving model to checkpoints\Optimization_Weights.365.hdf5
804/804 [==============================] - 1s 721us/step - loss: 0.5327 - accuracy: 0.7402
Epoch 366/500
804/804 [==============================] - 1s 732us/step - loss: 0.5326 - accuracy: 0.7404
Epoch 367/500
804/804 [==============================] - 1s 745us/step - loss: 0.5327 - accuracy: 0.7409
Epoch 368/500
804/804 [==============================] - 1s 728us/step - loss: 0.5326 - accuracy: 0.7410
Epoch 369/500
804/804 [==============================] - 1s 719us/step - loss: 0.5328 - accuracy: 0.7405
Epoch 370/500
773/80

804/804 [==============================] - 1s 716us/step - loss: 0.5323 - accuracy: 0.7409
Epoch 423/500
804/804 [==============================] - 1s 739us/step - loss: 0.5323 - accuracy: 0.7407
Epoch 424/500
804/804 [==============================] - 1s 704us/step - loss: 0.5323 - accuracy: 0.7410
Epoch 425/500
779/804 [============================>.] - ETA: 0s - loss: 0.5322 - accuracy: 0.7412
Epoch 425: saving model to checkpoints\Optimization_Weights.425.hdf5
804/804 [==============================] - 1s 715us/step - loss: 0.5325 - accuracy: 0.7409
Epoch 426/500
804/804 [==============================] - 1s 734us/step - loss: 0.5323 - accuracy: 0.7410
Epoch 427/500
804/804 [==============================] - 1s 754us/step - loss: 0.5323 - accuracy: 0.7406
Epoch 428/500
804/804 [==============================] - 1s 703us/step - loss: 0.5321 - accuracy: 0.7411
Epoch 429/500
804/804 [==============================] - 1s 705us/step - loss: 0.5322 - accuracy: 0.7414
Epoch 430/500
784/80

804/804 [==============================] - 1s 745us/step - loss: 0.5320 - accuracy: 0.7409
Epoch 483/500
804/804 [==============================] - 1s 871us/step - loss: 0.5320 - accuracy: 0.7402
Epoch 484/500
804/804 [==============================] - 1s 811us/step - loss: 0.5320 - accuracy: 0.7409
Epoch 485/500
774/804 [===========================>..] - ETA: 0s - loss: 0.5316 - accuracy: 0.7411
Epoch 485: saving model to checkpoints\Optimization_Weights.485.hdf5
804/804 [==============================] - 1s 792us/step - loss: 0.5322 - accuracy: 0.7405
Epoch 486/500
804/804 [==============================] - 1s 778us/step - loss: 0.5319 - accuracy: 0.7411
Epoch 487/500
804/804 [==============================] - 1s 757us/step - loss: 0.5319 - accuracy: 0.7407
Epoch 488/500
804/804 [==============================] - 1s 755us/step - loss: 0.5319 - accuracy: 0.7406
Epoch 489/500
804/804 [==============================] - 1s 745us/step - loss: 0.5321 - accuracy: 0.7409
Epoch 490/500
748/80

In [194]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_5.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5606 - accuracy: 0.7286 - 227ms/epoch - 849us/step
Loss: 0.560573935508728, Accuracy: 0.7286297082901001


In [195]:
# Export our model to HDF5 file
nn_5.save("AlphabetSoupCharity_Optimization.h5")